# Classificação: fake-news

## Introdução

In [ ]:
#@title

!pip -q install shap

import os
from functools import reduce

import numpy as np
import pandas as pd
import tensorflow as tf
import shap

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from google.colab import drive

In [ ]:
#@title

sns.set(palette=sns.color_palette("hls", 8))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

drive.mount('/content/drive')

## Coletando as informações

In [ ]:
DATASET = ('/content/drive/My Drive/Colab Notebooks/cs-no/'
           'datasets/572515_1037534_bundle_archive.zip')

In [ ]:
import zipfile
from sklearn.model_selection import train_test_split

with zipfile.ZipFile(DATASET) as z:
    z.extractall('./ds/')

t, f = (pd.read_csv('./ds/True.csv'),
        pd.read_csv('./ds/Fake.csv'))

def preprocess(t, f):
    t['target'] = 'true'
    f['target'] = 'fake'

    x = t.append(f)

    ds = x.date.str.strip()
    x['created_at'] = pd.to_datetime(ds, format='%B %d, %Y', errors='coerce')
    x.loc[x.created_at.isnull(), 'created_at'] = pd.to_datetime(ds[x.created_at.isnull()], format='%b %d, %Y', errors='coerce')
    x.loc[x.created_at.isnull(), 'created_at'] = pd.to_datetime(ds[x.created_at.isnull()], format='%d-%b-%y', errors='coerce')

    return x.dropna()

x = preprocess(t, f)

In [ ]:
def describe(x):
    print('samples:', len(x))
    print('features:', *x.columns)
    print('timeframe:', x.created_at.min().date(), x.created_at.max().date())

describe(x)

x.head(1)

## Definindo os intervalos de treinamento e teste

A separação entre treino e teste é muitas vezes realizada de forma aleatória.
É tão comum, que podemos observar utilitários para fazer isso no próprio sklearn (o [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)). O objetivo dessa estratégia é gerar uma divisão próxima à distribuição original.

Supondo que ambos subconjuntos produzidos sejam estatísticamente representativos em todas as suas características, eles apresentam o menor enviesamento e retenção de informação. Entretanto, isso pode gerar uma preocupação séria: estamos injetando informação do futuro para predizer o passado? Pense no seguinte cenário: Obama foi o presidente até 2016. Até então, haviam poucas menções ao atual presidente Trump (eleito em 2016) nas reportagens relacionadas à política.

Se nosso estimador se firmar em eventos pontuais (ex: palavras como "Obama" ou "Trump") para tomar suas decisões, podemos garantir que estes eventos irão ocorrer novamente? Como podemos avaliar a degradação de um modelo?

A separação temporal --- onde o passado é utilizado como treino e a informação mais recente é utilizada como teste --- também é uma estratégia de separação válida. Garantimos que o futuro não seja misturado com o passado e fortificamos o teste, sendo mais aderente com o cenário real.

In [ ]:
TEST_SIZE = .8
x = x.sort_values('created_at')

_at = int(len(x)*(1-TEST_SIZE))
train, test = x[:_at], x[_at:]

print(f'{len(train)} samples will be used for training.')
print('timeframe:', train.created_at.min().date(), train.created_at.max().date())
print('classes:', *zip(*np.unique(train.target, return_counts=True)), sep='\n')
print()

print(f'{len(test)} samples will be used for testing.')
print('timeframe:', test.created_at.min().date(), test.created_at.max().date())
print('classes:', *zip(*np.unique(test.target, return_counts=True)), sep='\n')

In [ ]:
plt.figure(figsize=(1, 2))
sns.barplot(*np.unique(train.target, return_counts=True));

## Distinguindo notícias falsas de verdadeiras a partir do texto

In [ ]:
from imblearn.over_sampling import RandomOverSampler

truth_s = RandomOverSampler()

z, y = np.asarray(train.text).reshape(-1, 1), train.target
z, y = truth_s.fit_resample(z, y)
z = z.ravel()

plt.figure(figsize=(1, 2))
sns.barplot(*np.unique(y, return_counts=True));

In [ ]:
from sklearn import metrics

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

truth_m = Pipeline([
    ('tfidf', TfidfVectorizer(strip_accents='unicode',
                              stop_words='english',
                              ngram_range=(1, 3),
                              max_features=4096,
                              lowercase=True)),
    ('lr', LogisticRegression())
])

truth_m.fit(z, y);

### Explicando o modelo

Há muitos coeficientes para serem explicados ao mesmo tempo:

In [ ]:
truth_tfidf = truth_m.named_steps['tfidf']
truth_lr = truth_m.named_steps['lr']

plt.figure(figsize=(5, 2))
sns.distplot(truth_lr.coef_);

In [ ]:
data = pd.DataFrame(truth_tfidf.transform(train.sample(1000).text).todense(),
                    columns=truth_tfidf.get_feature_names())

explainer = shap.LinearExplainer(truth_lr, data)
shap_values = explainer.shap_values(data)

In [ ]:
shap.summary_plot(shap_values, data)

### Avaliando modelo sobre o conjunto de teste

In [ ]:
p = truth_m.predict(test.text)

print('Classification Report:')
print(metrics.classification_report(test.target, p))

print('Report over missed samples')
miss = p != test.target
print(f'misses: {miss.sum()} ({miss.mean():.2%})',
      dict(zip(*np.unique(test.target[miss], return_counts=True))),
      sep='\n')

Olhar para as amostras que erramos pode nos ajudar a melhorar um modelo:

In [ ]:
test.loc[miss & (test.target == 'fake'), ['title', 'subject', 'target', 'created_at']][:20]

### Melhorando o modelo a partir de características adjuntas

Utilização de acentuação e caixa alta aconteceu em vários desses textos.
Não remover essas características, assim como as stop-words, pode melhorar o nosso modelo?

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'tfidf__lowercase': [True, False],
    'tfidf__stop_words': [None, 'english'],
    'tfidf__ngram_range': [(1, 1), (1, 3)],
    'tfidf__strip_accents': [None],
    'tfidf__max_features': [4096],
}

truth_g = GridSearchCV(truth_m, params,
                       cv=3,
                       n_jobs=-1,
                       verbose=2).fit(z, y)

In [ ]:
truth_g.best_params_

In [ ]:
truth_tfidf = truth_g.best_estimator_.named_steps['tfidf']
truth_lr = truth_g.best_estimator_.named_steps['lr']

data = pd.DataFrame(truth_tfidf.transform(train.sample(1000).text).todense(),
                    columns=truth_tfidf.get_feature_names())

explainer = shap.LinearExplainer(truth_lr, data)
shap_values = explainer.shap_values(data)

In [ ]:
shap.summary_plot(shap_values, data)

In [ ]:
p = truth_m.predict(test.text)

print('Classification Report:')
print(metrics.classification_report(test.target, p))

print('Report over missed samples')
miss = p != test.target
missed_fakes = test[miss & (test.target == 'fake')]

print(f'misses: {miss.sum()} ({miss.mean():.2%})',
      dict(zip(*np.unique(test.target[miss], return_counts=True))),
      sep='\n')

In [ ]:
missed_fakes.drop(columns='text')[:20]

In [ ]:
data = pd.DataFrame(truth_tfidf.transform(missed_fakes.text).todense(),
                    columns=truth_tfidf.get_feature_names())

explainer = shap.LinearExplainer(truth_lr, data)
shap_values = explainer.shap_values(data)

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[:20], data[:20])